In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import_ipynb module 설치
!pip install import_ipynb

!apt-get update
!apt-get -qq install xxd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.0 MB/s 
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/d

In [ ]:
# import를 위한 경로이동
%cd /content/drive/MyDrive/team_malmungchi/colab/speaker_verification/code
!ls

/content/drive/.shortcut-targets-by-id/1rEYIox5uYM9uP2spDttCYCqtt2GvSbn1/team_malmungchi/colab/speaker_verification/code
batcher.ipynb		 constants.ipynb     fitter.ipynb      test_eer.ipynb
call_development1.ipynb  convert.ipynb	     loss.ipynb        utils.ipynb
call_development2.ipynb  development.ipynb   network.ipynb
call_development3.ipynb  eval_metrics.ipynb  preprocess.ipynb
call_tflite.ipynb	 evaluation.ipynb    temp.wav


In [ ]:
import tensorflow as tf
from tensorflow import keras
import pickle
import numpy as np

import import_ipynb
import network
from utils import load_best_checkpoint

importing Jupyter notebook from network.ipynb
importing Jupyter notebook from eval_metrics.ipynb
importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from constants.ipynb


In [ ]:
def convert(model_name, checkpoint_dir, model_path, use_LSTM=False):
  Model = network.get_network(model_name)

  best_ckpt = load_best_checkpoint(checkpoint_dir)
  Model.load_weights(checkpoint_dir+'/'+best_ckpt)

  if use_LSTM:
    # lstm의 경우 특별히 명시해줘야 하는것이 있다.
    # https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/examples/experimental_new_converter/Keras_LSTM_fusion_Codelab.ipynb#scrollTo=tB1NZBUHDogR
    run_model = tf.function(lambda x: Model(x))
    BATCH_SIZE = 1
    STEPS = 49
    INPUT_SIZE = 40
    concrete_func = run_model.get_concrete_function(
      tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], Model.inputs[0].dtype))

    # model directory.
    MODEL_DIR = "lstm_tmp"
    Model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

    converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
  else: converter = tf.lite.TFLiteConverter.from_keras_model(Model)

  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  #converter._experimental_default_to_single_batch_in_tensor_list_ops = True # lstm의 경우 배치 크기가 1임을 명시해줘야한다.
  
  # input/output layer까지 int8로 양자화하려면 representative dataset이 필요하다.
  with open('../data/dataset/train_cpp_1581_200_49_40.pickle',"rb") as f:
    X = pickle.load(f)
    X = np.concatenate(X)
    X = X.astype(np.float32)
  def representative_data_gen():
    for input_value in X:
      yield [input_value]
  converter.representative_dataset = representative_data_gen
  converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
  converter.inference_input_type = tf.int8
  converter.inference_output_type = tf.int8

  tflite_model = converter.convert()
  open(model_path+"/"+checkpoint_dir.split('/')[-1]+".tflite", "wb").write(tflite_model)

In [ ]:
convert('arduino_CRNN','../model/arduino_CRNN-naive_batcher-cross_entropy', '../model', use_LSTM=True)

In [ ]:
%cd ..
%cd model
!ls

/content/drive/.shortcut-targets-by-id/1rEYIox5uYM9uP2spDttCYCqtt2GvSbn1/team_malmungchi/colab/speaker_verification
/content/drive/.shortcut-targets-by-id/1rEYIox5uYM9uP2spDttCYCqtt2GvSbn1/team_malmungchi/colab/speaker_verification/model
ACRNN-naive_batcher-cross_entropy
ACRNN-simMat_batcher-simMat_loss
ACRNN-simMat_batcher-simMat_loss--transferedFrom--ACRNN-naive_batcher-cross_entropy
ADRNN-naive_batcher-cross_entropy
ADRNN-simMat_batcher-simMat_loss
ADRNN-simMat_batcher-simMat_loss-931-large--transferedFrom--ADRNN-naive_batcher-cross_entropy
ADRNN-simMat_batcher-simMat_loss-931--transferedFrom--ADRNN-naive_batcher-cross_entropy
ADRNN-simMat_batcher-simMat_loss--transferedFrom--ADRNN-naive_batcher-cross_entropy
arduino_CNN-naive_batcher-cross_entropy
arduino_CNN-naive_batcher-cross_entropy.cc
arduino_CNN-naive_batcher-cross_entropy-noCustom.cc
arduino_CNN-naive_batcher-cross_entropy-noCustom.tflite
arduino_CNN-naive_batcher-cross_entropy.tflite
arduino_CRNN-naive_batcher-cross_entropy

In [ ]:
!xxd -i arduino_CRNN-naive_batcher-cross_entropy.tflite > arduino_CRNN-naive_batcher-cross_entropy.cc